# Running the thermodynamic analysis using the functions of the pipeline

Each model resulting from the first analysis meking use of GapFilling for finding new functionalities has a different combination of reactions allowing the substrate to be used for growth and/or be converted into the target compound. Hence, the list of reactions involved in the transformations changes among the variants. Thus the pipeline uses pathway length and thermodynamic driving force as two parameters for the evaluation of the strain designs. In this pipeline the MDF value [1] of a pathway is used to determine its thermodynamic feasibility. The aim of this module (Figure 1, step iii) is to first  identify which reactions are involved in the pathway and then calculate the pathway’s Max-min driving force (MDF) value. The thermidynamic analysis done by the pipeline make use of a the Python package [equilibrator-pathway](https://gitlab.com/equilibrator/equilibrator-pathway) for MDF calculation. This package is still under development and the the module of the pipeline reliying on it is the one that still needs further improvements. 

The functions for thermodynamic analysis are called within the analysis module by the function *cons_prod_dict*. In the following tutorial some of the functions for the thermodynamic analysis are used individually on one of the combination of the strains of *E. coli* growing on methane. 

The logic of the analysis is the following:
1.	Add the reactions for growth on methane
2.	Check if it grows on methane
3.	Check if it produce the target (i.e. itacon)
4.  Use the pipeline function for the rest of the analusis


In [1]:
import cobra

from cobra.manipulation import modify 
import csv
from pipeline.scripts.input_parser import *
from pipeline.scripts.import_models import get_universal_main, get_reference_model
from pipeline.scripts.analysis import *
from pipeline.scripts.path_definition_mdf import whole_procedure_path_definition, mdf_analysis


In [2]:
data_repo = "./pipeline/inputs"
model = get_reference_model(data_repo, './pipeline/inputs/ecoli_tutorial.csv')
universal = get_universal_main(data_repo, './pipeline/inputs/ecoli_tutorial.csv')

Using license file C:\Users\delip\gurobi.lic
Academic license - for non-commercial use only


In [3]:
main('./pipeline/inputs/ecoli_tutorial.csv', universal, model)

For ch4 there isn't any uptake trasnsporter in the reference model
The trasporter has been added to the 
                                reference model from the input file
For nh4 there isn't any uptake trasnsporter in the reference model

For nh4 there is a transport reaction in the universal model for the uptake from the extracellular space:  
Reaction ID:  NH4t 
Reaction equation:  nh4_e <=> nh4_c

For nh4 there is a transport reaction 
                                    in the universal model for the uptake 
                                    from the periplasm:  
Reaction ID:  NH4t4pp 
Reaction equation:  k_c + nh4_p --> k_p + nh4_c 


For nh4 there is a transport reaction 
                                    in the universal model for the uptake 
                                    from the periplasm:  
Reaction ID:  NH4tpp 
Reaction equation:  nh4_p <=> nh4_c 


For nh4 there is a transport reaction 
                                    in the universal model for the uptake 
 

Name,iML1515
Memory address,0x017ca54b6828
Number of metabolites,1877
Number of reactions,2716
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [4]:
model.reactions.EX_ch4_e

Reaction identifier,EX_ch4_e
Name,EX ch4 LPAREN e RPAREN
Memory address,0x017cae39f898
Stoichiometry,ch4_e --> Methane -->
GPR,
Lower bound,0.0
Upper bound,1000.0


## 1. Add reactions needed for growth on methane

In [5]:
mmo = universal.reactions.R01143
model.add_reaction(mmo)

In [6]:
model.reactions.R01143

Reaction identifier,R01143
Name,1 H(+) + 1 dioxygen + 1 NADPH + 1 methane = 1 methanol + 1 H2O + 1 NADP(+)
Memory address,0x017cb179a780
Stoichiometry,ch4_c + h_c + nadph_c + o2_c <=> h2o_c + meoh_c + nadp_c Methane + H+ + Nicotinamide adenine dinucleotide phosphate - reduced + O2 O2 <=> H2O H2O + Methanol + Nicotinamide adenine dinucleotide phosphate
GPR,MetaCyc_GJFB_518_MONOMER_i or MetaCyc_GJFB_519_MONOMER_i or MetaCyc_GJFB_520_MONOMER_i
Lower bound,-1000
Upper bound,1000


In [7]:
alcd1 = universal.reactions.ALCD1
model.add_reaction(alcd1)

In [8]:
model.reactions.ALCD1

Reaction identifier,ALCD1
Name,Alcohol dehydrogenase (methanol)
Memory address,0x017cb0843828
Stoichiometry,meoh_c + nad_c <=> fald_c + h_c + nadh_c Methanol + Nicotinamide adenine dinucleotide <=> Formaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## 2. Check if it indeed grows on methane

In [9]:
print(model.objective.expression)

1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685


In [10]:
from cobra.flux_analysis import phenotype_phase_plane
from cobra.flux_analysis import production_envelope

In [11]:
metex = model.reactions.EX_ch4_e
metex.bounds 

(0.0, 1000.0)

In [12]:
metex.lower_bound = -1000

In [13]:
glc = model.reactions.EX_glc__D_e
glc.lower_bound = 0

In [14]:
biomass = get_biomass_equation(model)

In [15]:
biomass.upper_bound = 0.877
biomass.lower_bound = 0.04385

In [16]:
fba_check = model.optimize()
print(fba_check.objective_value, '\n', fba_check.fluxes['EX_ch4_e'])

0.877 
 -225.59855453792775


In [17]:
for i in model.reactions:
    if i.flux <= -10 and 'EX_' in i.id: 
        print(i.id, i.reaction, i.flux)

EX_ch4_e ch4_e <=>  -225.59855453792775
EX_o2_e o2_e <=>  -413.3287546663555


It does consume as much methane as possible!

## 3. Does it produce the target?

In [18]:
fba_check.fluxes['EX_lac__L_e']

0.0

In [19]:
lacex = model.reactions.get_by_id('EX_lac__L_e')
lacex.lower_bound = 2

In [20]:
fba_check_production = model.optimize()
print(fba_check_production.objective_value, '\n', fba_check_production.fluxes['EX_ch4_e'])

0.877 
 -258.66522120459376


In [21]:
fba_check_production.fluxes['EX_lac__L_e']

2.0

##### It does not produce lactate while the objective of the model is the biomass reaction but it produces lactate when its exchange reaction is set to be the objective of the model's optimization

## 4. Use the pipelines function for the rest of the analysis
These two functions *whole_procedure_path_definition* and *mdf_analysis* are used. The former prepares for the analysis of MDF. The latter makes use of the function from [equilibrator-pathway](https://gitlab.com/equilibrator/equilibrator-pathway) package to calculate MDF. More details on the functionaliies below.

*whole_procedure_path_definition* Calls the functions involved in finding the minimal set of reactions for the production of the target, in particular:
1. adds the free balancing reactions
2. remove maintenance
3. constrains substrate and product using the stoichiometric
    coefficients
4. checks the carbon source to be the expected one
5. set the target as model objective
6. checks the metabolism
7. gets reactions list from pFBA 
8. processes the raw list to get the minimal reaction set
9. generates SBtab file with the results ready for MDF calculation

*mdf_analysis* Salls the function for the identification fo the minimal
1. set of reactions active in the conversion. 
2. uses the gneerated SBtab for mdf calculation

In [23]:
pruned_path = whole_procedure_path_definition('./pipeline/inputs/ecoli_tutorial.csv', model, '{}_Run_{}.tsv'.format('lac', '9-10'), 'input_mdf_{}_Run_{}.tsv'.format('lac', '9-10'))
mdf_value, path_length = mdf_analysis('./pipeline/inputs/ecoli_tutorial.csv', model, '{}_Run_{}.tsv'.format('lac', '9-10'), 'input_mdf_{}_Run_{}.tsv'.format('lac', '9-10'))
thermodynamic = {} 
if mdf_value == None:
    thermodynamic['mdf'] = mdf_value
else:
    thermodynamic['mdf'] = mdf_value.magnitude
thermodynamic['pathway_langth'] = path_length

Substrate's exchange reaction bounds :  (-6.6000000000000005, -5.4)
Target's exchange reaction bounds :  (2.0, 1000.0)
Carbon source:  []
Target:  EX_lac__L_e: lac__L_e --> 
FBA objective value:  None 
Substrate consumption flux:  -6.6000000000000005 
Target production flux:  37.81615397732563 
Biomass:  0.04385 

pFBA is infeasible, control if the coefficients of the 
        reaction equation are correct (or use a different boudary 
        reaction of the target as model objective):  None (infeasible)
The thermodynamic analysis cannot proceed because of infeasible pFBA
Substrate's exchange reaction bounds :  (-6.6000000000000005, -5.4)
Target's exchange reaction bounds :  (2.0, 1000.0)
Carbon source:  []
Target:  EX_lac__L_e: lac__L_e --> 
FBA objective value:  None 
Substrate consumption flux:  -6.6000000000000005 
Target production flux:  32.40494097019504 
Biomass:  0.04385 

pFBA is infeasible, control if the coefficients of the 
        reaction equation are correct (or use a d

In [24]:
thermodynamic

{'mdf': None, 'pathway_langth': None}

In [25]:
for i in pruned_path:
    print(i.id, i.reaction, i.bounds, i.flux)

TypeError: 'NoneType' object is not iterable

In [26]:
print(model.objective.expression)

1.0*EX_lac__L_e - 1.0*EX_lac__L_e_reverse_8586b


<h1 style="color: red;"> TODO </h1>

<h4 style="color: blue;"> Debug: why does it have FBA obj value = None when the previous FBA calculation give a value? Check the thermodynamic module. </h4>

## Conclusions

This module has not be fully debugged yet, since the strategy used is not the most ideal one, but is one of the more easily available at the moment in which the pipeline has been written. The identification and evaluation of pathways could be facilitated by an algorithm able to enumerate all the possible pathways for a desired phenotype (e.g production of L-lactate from *E. coli* strain assimilating methane) and calculate their MDF values. This is exactly what [OptMDFpathway](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1006492) does [2]. This program has been implemented as function in [CellNetAnalyzer toolbox](https://www2.mpi-magdeburg.mpg.de/projects/cna/cna.html) in Matlab [3]. CellNetAnalyser toolbox is being [rewritten in Python](https://github.com/ARB-Lab/CNApy), but the OptMDFpathway function still has to be implemented. Once that function will be available the pipeline will make use of it in substitution to the pFBA-MDF approach.

##### References
1. E. Noor, A. Bar-Even, A. Flamholz, E. Reznik, W. Liebermeister, and R. Milo, “Pathway Thermodynamics Highlights Kinetic Obstacles in Central Metabolism,” PLoS Comput. Biol., vol. 10, no. 2, p. e1003483, Feb. 2014, doi: 10.1371/journal.pcbi.1003483
2. O. Hädicke, A. von Kamp, T. Aydogan, and S. Klamt, “OptMDFpathway: Identification of metabolic pathways with maximal thermodynamic driving force and its application for analyzing the endogenous CO2 fixation potential of Escherichia coli,” PLoS Comput. Biol., vol. 14, no. 9, p. e1006492, Sep. 2018, doi: 10.1371/journal.pcbi.1006492.
3. A. von Kamp, S. Thiele, O. Hädicke, and S. Klamt, “Use of CellNetAnalyzer in biotechnology and metabolic engineering,” Journal of Biotechnology, vol. 261. Elsevier B.V., pp. 221–228, Nov. 10, 2017, doi: 10.1016/j.jbiotec.2017.05.001.